# KME Timing Experiments

This notebook performs a basic timing experiment to compare KME runtime with computing the signature kernel.

In [1]:
using MAT
using Statistics
include("compute_features.jl")
include("PathSignatures.jl")
include("regression_utils.jl")
include("utils.jl")

SW_MMD_kernel_from_SWD (generic function with 3 methods)

In [2]:
Threads.nthreads()

1

In [3]:
N_all = [10, 20, 30, 40] # Number of simulations
T_all = [5, 10, 15, 20] # Number of time steps

4-element Vector{Int64}:
  5
 10
 15
 20

In [4]:
# Load persistence diagrams from ss=50, init100
base_fpath = "data/ss50/init100/"
fname = string(base_fpath, "PD/PD.mat")
file = matopen(fname, "r")
B0 = read(file, "B0")
B1 = read(file, "B1")
B2 = read(file, "B2")
close(file)


In [9]:
SWD_time = zeros(4,4)

for (i, N) in enumerate(N_all)
    for (j, T) in enumerate(T_all)
        start = time()
        D_all = batch_SWD(B0, B1, B2, 1, N, T)
        SWD_time[i,j] = time() - start
    end
end


In [10]:
SWD_time

4×4 Matrix{Float64}:
 0.12536   0.246593  0.534773   0.940456
 0.242112  0.925917  2.04854    3.64619
 0.534784  2.04885   4.77143    8.05156
 0.956096  3.64669   7.97287   14.0819

In [11]:
MK_time = zeros(4,4)
moment_level = 2
signature_level = 3

for (i, N) in enumerate(N_all)
    for (j, T) in enumerate(T_all)
        start = time()
        curB0 = B0[1:N]
        curB1 = B1[1:N]
        curB2 = B2[1:N]

        for n = 1:N
            curB0[n] = curB0[n][1:T]
            curB1[n] = curB1[n][1:T]
            curB2[n] = curB2[n][1:T]
        end

        FT = compute_PD_moments(curB0, curB1, curB2, moment_level, "", false)
        K = dsignature_kernel_matrix(FT, [], signature_level, "R")

        MK_time[i,j] = time() - start
    end
end

In [12]:
MK_time

4×4 Matrix{Float64}:
 0.00252795  0.00213599  0.00302482  0.00716305
 0.00548816  0.00687194  0.0139999   0.014925
 0.0110071   0.012964    0.0158989   0.0350108
 0.026649    0.021143    0.024406    0.047616

In [13]:
MK_time[1,2]

0.0021359920501708984